In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
credits_raw = pd.read_csv("./data/credits.csv")
credits_raw.set_axis(
    ["id_persona","id_titulo","nombre","nombre_personaje","rol"],axis=1,inplace=True)
credits_raw.head()

,id_persona,id_titulo,nombre,nombre_personaje,rol
0,59401,ts20945,Joe Besser,Joe,ACTOR
1,31460,ts20945,Moe Howard,Moe,ACTOR
2,31461,ts20945,Larry Fine,Larry,ACTOR
3,21174,tm19248,Buster Keaton,Johnny Gray,ACTOR
4,28713,tm19248,Marion Mack,Annabelle Lee,ACTOR


In [3]:
titles_raw = pd.read_csv("./data/titles.csv")
titles_raw.set_axis(
    ["id_titulo","nombre","tipo","descripción","año_creación","clasificación"\
     ,"duración","generos","pais/es productores","temporadas","id_imdb"\
     ,"calificación_imdb","votos_imdb","popularidad_tmdb","score_tmdb"]
    ,axis=1,inplace=True)

In [4]:
credits_raw.nombre.value_counts()

George 'Gabby' Hayes    49
Roy Rogers              45
Bess Flowers            44
Joseph Kane             41
Gene Autry              40
                        ..
Seigi Ozeki              1
Winai Kraibutr           1
Buakaw Banchamek         1
Thammaros Jaichuen       1
Ricky Lomax              1
Name: nombre, Length: 79758, dtype: int64

In [168]:
persona = credits_raw[["id_persona","nombre","rol"]]
persona = ppl.drop_duplicates()
persona.rol = ppl.rol.str.lower()
persona.head()

,id_persona,nombre,rol
0,59401,Joe Besser,actor
1,31460,Moe Howard,actor
2,31461,Larry Fine,actor
3,21174,Buster Keaton,actor
4,28713,Marion Mack,actor


In [169]:
titulo = titles_raw[["id_titulo","nombre", 'tipo']].copy()
titulo.tipo = titulo.tipo.str.lower()

titulo.head()

,id_titulo,nombre,tipo
0,ts20945,The Three Stooges,show
1,tm19248,The General,movie
2,tm82253,The Best Years of Our Lives,movie
3,tm83884,His Girl Friday,movie
4,tm56584,In a Lonely Place,movie


## Nodos

In [188]:
nodes = [persona, titulo]
table_names = ['persona', 'titulo']

for idx, df in enumerate(nodes):

    label = table_names[idx]
    colnames = df.columns
    dfcopy = df.set_axis(['id', 'nombre', 'tipo'], axis=1)
    dfcopy.nombre = dfcopy.nombre.str.replace("\'", "\\'")
    execution_commands = []

    for _, entry in dfcopy.iterrows():
        
        s1 = f"create (p:{label}:{entry.tipo} "
        s2 = f"{{{colnames[0]}:'{entry.id}', {colnames[1]}:'{entry.nombre}'}})"
        create_statement = s1 + s2
        execution_commands.append(create_statement)


    def execute_transactions(execution_commands):
        data_base_connection = GraphDatabase.driver(
            uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
        session = data_base_connection.session()    
        for i in execution_commands:
            session.run(i)

    execute_transactions(execution_commands)

In [191]:
# Create statement for ACTORES
"create (p:persona:actor { id_persona:'_id_persona_', nombre:'_nombre_'})"
# Create statement for DIRECTORES
"create (p:persona:DIRECTOR { id_persona:'_id_persona_', nombre:'_nombre_director'})"
# Create statement for TITULOS
"create (t:titulo:tipo { id_titulo:'_id_titulo_', nombre:'_nombre_'})"


''

## Relaciones

In [193]:
trabaja = credits_raw[["id_titulo","id_persona","nombre_personaje","rol"]].copy()
trabaja.rol = trabaja.rol.str.lower()
trabaja.head()

,id_titulo,id_persona,nombre_personaje,rol
0,ts20945,59401,Joe,actor
1,ts20945,31460,Moe,actor
2,ts20945,31461,Larry,actor
3,tm19248,21174,Johnny Gray,actor
4,tm19248,28713,Annabelle Lee,actor


In [204]:
rels = [trabaja]
table_names = ['trabaja_en']

for idx, df in enumerate(rels):

    label = table_names[idx]
    colnames = df.columns
    dfcopy = df.set_axis(['titulo', 'persona', 'personaje', 'rol'], axis=1)
    dfcopy.personaje = dfcopy.personaje.str.replace("\'", "\\'")
    execution_commands = []

    for _, entry in dfcopy.iterrows():
        
        s1 = f"match (t {{{colnames[0]}:'{entry.titulo}'}}) "
        s2 = f"match (p:{entry.rol} {{{colnames[1]}:'{entry.persona}'}}) "
        s3 = f"create (p)-[te:{label}]->(t)"
        
        create_statement = s1 + s2 + s3
        
        if type(entry.personaje) == str:
            s4 = f" set te.{colnames[2]} = '{entry.personaje}'"
            create_statement += s4
            
        execution_commands.append(create_statement)


    def execute_transactions(execution_commands):
        data_base_connection = GraphDatabase.driver(
            uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
        session = data_base_connection.session()    
        for i in execution_commands:
            session.run(i)

            
    #for elem in execution_commands[:5]:
    #   print(elem)
    execute_transactions(execution_commands)

In [202]:
# Create statement for relationship TRABAJA without character name
wo = "\
match (t {id_titulo:'_id_titulo_'}) \
match (p:rol {id_persona:'_id_persona_'}) \
create (p)-[te:trabaja_en]->(t)"
# Create statement for relationship TRABAJA with character name
w = wo + " \
set te.nombre_personaje = '_nombre_personaje_'"

"match (ttitulo {id_titulo:'_id_titulo_'}) match (p:rol {id_persona:'_id_persona_'}) create (p)-[te:trabaja_en]->(t)"